# Homework 8

## CSCI E-82A


In the a previous homework assignments, you used two different dynamic programming algorithms and Monte Carlo reinforcement learning to solve a robot navigation problem by finding optimal paths to a goal in a simplified warehouse environment. Now you will use time differencing reinforcement learning to find optimal paths in the same environment.

The configuration of the warehouse environment is illustrated in the figure below.

<img src="GridWorldFactory.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **Grid World for Factory Navigation Example** </center>

The goal is for the robot to deliver some material to position (state) 12, shown in blue. Since there is a goal state or **terminal state** this an **episodic task**. 

There are some barriers comprised of the states $\{ 6, 7, 8 \}$ and $\{ 16, 17, 18 \}$, shown with hash marks. In a real warehouse, these positions might be occupied by shelving or equipment. We do not want the robot to hit these barriers. Thus, we say that transitioning to these barrier states is **taboo**.

As before, we do not want the robot to hit the edges of the grid world, which represent the outer walls of the warehouse. 

## Representation

You are, no doubt, familiar with the representation for this problem by now.    

As with many such problems, the starting place is creating the **representation**. In the cell below encode your representation for the possible action-state transitions. From each state there are 4 possible actions:
- up, u
- down, d,
- left, l
- right, r

There are a few special cases you need to consider:
- Any action transitioning state off the grid or into a barrier should keep the state unchanged. 
- Any action in the goal state keeps the state unchanged. 
- Any transition within the taboo (barrier) states can keep the state unchanged. If you experiment, you will see that other encodings work as well since the value of a barrier states are always zero and there are no actions transitioning into these states. 

> **Hint:** It may help you create a pencil and paper sketch of the transitions, rewards, and probabilities or policy. This can help you to keep the bookkeeping correct. 

In [1]:
## import numpy for latter
import numpy as np
import numpy.random as nr



You need to define the initial transition probabilities for the Markov process. Set the probabilities for each transition as a **uniform distribution** leading to random action by the robot. 

> **Note:** As these are just starting values, the exact values of the transition probabilities are not actually all that important in terms of solving the RL problem. Also, notice that it does not matter how the taboo state transitions are encoded. The point of the DP algorithm is to learn the transition policy.

The robot receives the following rewards:
- 10 for entering position 0. 
- -1 for attempting to leave the grid. In other words, we penalize the robot for hitting the edges of the grid.  
- -0.1 for all other state transitions, which is the cost for the robot to move from one state to another. If we did not have this penalty, the robot could follow any random plan to the goal which did not hit the edges. 

This **reward structure is unknown to the MC RL agent**. The agent must **learn** the rewards by sampling the environment. 

In the code cell below encode your representation of this reward structure you will use in your simulated environment.  

You will find it useful to create a list of taboo states, which you can encode in the cell below.

## TD(0) Policy Evaluation

With your representations defined, you can now create and test functions to perform TD(0) **policy evaluation**. 

As a first step you will need a function to find the rewards and next state given a state and an action. You are welcome to start with the `state_values` function from the TD/Q-learning notebook. However, keep in mind that you must modify this code to correctly treat the taboo states of the barrier. Specifically, taboo states should not be visited. 

Execute your code to test it for each possible action from state 11.  

Examine your results. Are the action values consistent with the transitions?

ANS: 

Next, you need to create a function to compute the state values using the TD(0) algorithm. You should use the function you just created  to find the rewards and next state given a state and action. You are welcome to use the `td_0_state_values` function from the TD/Q-learning notebook as a starting point.  

Execute your function for 1,000 episodes and examine the results.

Examine your results and answer the following questions to ensure you action value function operates correctly:
1. Are the values of the taboo states 0? ANS:
2. Are the states with the highest values adjacent to the terminal state? ANS: 
3. Are the values of the states decreasing as the distance from the terminal state increases? ANS: 


## SARSA(0) Policy Improvement

Now you will perform policy improvement using the SARSA(0) algorithm.  You are welcome to start with the `select_a_prime` and `SARSA_0` functions from the TD/Q-learning notebooks.    

Execute your code for 1,000 episodes, and with $\alpha = 0.2$, and $\epsilon = 0.1$)

Examine the action values you have computed. Ensure that the action values are 0 for the goal and taboo states. Also check that the actions with the largest values for each state make sense in terms of reaching the goal. 

With the action value function completed, you will now create and test code to perform GPI with SARSA(0).  You are welcome to use the `SRASA_0_GPI` function from the TD/Q-learning notebook as a starting point. 

Execute your code for 10 cycles of 100 episodes, with $\alpha = 0.2$, $\gamma = 0.9$ and $\epsilon = 0.01$, and examine the results.

Verify that your results make sense? For example, starting at state 2 or 22, do the most probable actions follow a shortest path?

ANS: 

## Apply Double Q-Learning

As a next step, you will apply Double Q-learning(0) to the warehouse navigation problem. In the cell below create and test a function to perform Double Q-Learning for this problem. You are welcome to use the `double_Q_learning` function from the TD/Q-learning notebook as a starting point.

Execute your code for 10 cycles of 500 episodes, with $\alpha = 0.2$, and $\gamma = 0.9$ and examine the results.

Examine the action values you have computed. Ensure that the action values are 0 for the goal and taboo states. Also check that the actions with the largest values for each state make sense in terms of reaching the goal. 

With the action value function completed, you will now create and test code to perform GPI with Double Q-Learning(0).  You are welcome to use the `double_Q_learning_0_GPI` function from the TD/Q-learning notebook as a starting point. 

Execute your code for 10 cycles of 500 episodes, with $\alpha = 0.2$, $\gamma = 0.9$ and $\epsilon = 0.01$, and examine the results. 

Verify that your results make sense? For example, starting at state 2 or 22, do the most probable actions follow a shortest path?

ANS: 

## N-Step TD Learning

Finally, you will apply N-Step TD learning and N-Step SARSA to the warehouse navigation problem.  First create a function to perform N-step TD policy evaluation. You are welcome to start with the `TD_n` policy evaluation function from the TD/Q-Learning notebook. 

Test your function using 1,000 episodes, $n = 4$, $\gamma = 0.9$, and $\alpha = 0.2$.

Verify that the result you obtained appears correct. Are the values of the goal and taboo states all 0? Do the state values decrease with distance from the goal?

Now that you have an estimate of the best values for the number of steps and the learning rate you can compute the action values using multi-step SARSA. In the cell below, create and test a function to compute the action values using N-step SARSA. You are welcome to use the `SRARSA_n` function from the TD/Q-learning notebook as a starting point. 

Test your function by executing 4-step SARSA for 1,000 episodes with $\alpha = 0.2$ and $\gamma = 0.9$ and using the optimum number of steps and learning rate you have determined. 

Verify that the results you have computed appear correct using the aforementioned criteria. 

Finally, create a function to use the GPI algorithm with N-step SARSA in the cell below. You are welcome to start with the `SARSA_n_GPI` function from the TD/Q-learning notebook. 

Execute your function using 4 step SARSA for 5 cycles of 500 episodes, with $\alpha = 0.2$, $\epsilon = 0.1$, and $\gamma = 0.9$.

Examine your results. Verify that the most probable paths to the goal from states 2 and 22 are the shortest possible.  